# Project 4: Wrangle, assess, clean and analyse twitter data - WeRateDogs


                            Christine Shuttleworth, 1st of October 2020



### Table of Contents
- [Introduction](#intro)
- [Part I - Data wrangling](#wrangling)
    - [Twitter Archive - load csv file](#load_csv)
    - [Twitter API - access and load data via Twitter API access](#twitter_api)
    - [Download and ingest neural network predictor data using requests](#requests)
- [Part II - Assess data](#assess)
    - [Visual assessment: data overview](#visual)
    - [Programmatic assessment:](#programmatic)
        - [Data structure:](#structure)
        - [Data quality:](#quality)
    - [Summary list of data issues:](#summary_issues)
        - [Tidyness issues:](#tidyness)
        - [Cleanliness issues:](#cleanliness)
- [Part III - Clean data and create twitter_archive_master.csv file](#clean)
    - [Data tidyness issues:](#tidy)
        - [Define, code and test issue: In_reply_to_status, in_reply_to_user_id](#def1)
        - [Define, code and test issues: Find best dog type prediction and merge this, the img_num and image_url column with df_ta dataframe. Inconsistent dog names.](#def2)
        - [Define, code and test issue: Merge df_json columns with df_ta dataframe on the tweed_id column. Delete df_json dataframe.](#def3)
        - [Define, code and test issue: Doggo, Floofer, Pupper, Puppo columns need to be melted into one column - dog_age_category](#def4)
        - [Define, code and test issue: Text column should only contain text, not Hashtag, ratings and short URL](#def5)
    - [Data quality issues:](#quality)
        - [Define, code and test issues: Wrong data types and extract short source name and save as categorical data](#def1)
        - [Define, code and test issues: Suspicious rating_numerators and denominator values](#defq2)
        - [Define, code and test issue: Compare hashtag data from json source and from text column, add missing values to df_ta](#defq3)
        - [Code issue: 4](#codeq3)
        - [Test issue: 4](#testq3)
        - [Define issue 4 +8: ](#defq4)
        - [Code issue 4 + 8:](#codeq4)
        - [Test issue: 4 + 8](#testq4)
        - [Define issue 5: ](#defq5)
        - [Code issue 5:](#codeq5)
        - [Test issue: 5](#testq5)

- [Part IV - Analyse data](#clean)
    - [Insight 1: x](#insight1) Which type of dog is rated the most often and the highest?
    - [Insight 2: x](#insight2)
    - [Insight 3: x](#insight3)





<a id='intro'></a>
### Introduction 

For this report, I wrangled WeRateDogs Twitter data to create interesting and trustworthy data insights and visualizations of the dog rating twitter feed. 

The twitter data will be enhanced with information of likely breed of the dog being rated, based on images available in the tweets. This data originates from a neural network image prediction data set of types of dogs.

To achieve this, I createe a solid and clean master dataset. Possible questions to ask:
- Which dog type is being rated the most often and the hightest?

Based on the analysis I created two reports:

    wrangle_report.pdf - summary of my wrangling effort
    act_report.pdf - insights and visualisation of the findings as a magazine article or blog post

<a id='wrangling'></a>
### Part 1 - Data wrangling

Set up python environment

In [199]:
import pandas as pd
import numpy as np
import tweepy as tw
import requests
import config as cfg
import os
from pathlib import Path 
import json
from dotenv import load_dotenv

%matplotlib inline
#%load_ext dotenv
#%dotenv

pd.options.display.max_rows = 999
pd.options.display.max_colwidth=500

<a id='load_csv'></a>
#### Load twitter_archive_enhanced.csv and learn about the data

In [143]:
df_ta = pd.read_csv('twitter-archive-enhanced.csv')

In [144]:
df_ta.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   tweet_id                    2356 non-null   int64  
 1   in_reply_to_status_id       78 non-null     float64
 2   in_reply_to_user_id         78 non-null     float64
 3   timestamp                   2356 non-null   object 
 4   source                      2356 non-null   object 
 5   text                        2356 non-null   object 
 6   retweeted_status_id         181 non-null    float64
 7   retweeted_status_user_id    181 non-null    float64
 8   retweeted_status_timestamp  181 non-null    object 
 9   expanded_urls               2297 non-null   object 
 10  rating_numerator            2356 non-null   int64  
 11  rating_denominator          2356 non-null   int64  
 12  name                        2356 non-null   object 
 13  doggo                       2356 

Columns:
1. tweet_id: twitter reference for this particular tweet
2. in_reply_to_status_id: twitter_id of tweet that was replied to. Tweets with NaN in this column are original tweets.
3. in_reply_to_user_id: user_id who wrote the reply 
4. timestamp: timestamp of the tweet
5. source: source of tweet - Twitter for iPhone, Vine - Make a Scene, Twitter Web Client, TweetDeck   
6. text: text of tweet: with hashtags and URL link to tweet.
7. retweeted_status_id: twitter_id of tweet that retweeted original tweet to. Tweets with NaN in this column were not retweeted.
8. retweeted_status_user_id: user_id who retweeted
9. retweeted_status_timestamp: timestamp of the retweet 
10. expanded_urls: full URL of the original tweet                
11. rating_numerator: rating of dog ...
12. rating_denominator: ... out of this number   
13. name: dog name   
14. doggo: flag if this dog falls into the doggo category
15. floofer: flag if this dog falls into the doggo category
16. pupper: flag if this dog falls into the doggo category
17. puppo: flag if this dog falls into the doggo category 

In [145]:
df_ta.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None


In [146]:
df_ta.source.value_counts()

<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>     2221
<a href="http://vine.co" rel="nofollow">Vine - Make a Scene</a>                          91
<a href="http://twitter.com" rel="nofollow">Twitter Web Client</a>                       33
<a href="https://about.twitter.com/products/tweetdeck" rel="nofollow">TweetDeck</a>      11
Name: source, dtype: int64

In [147]:
#df_ta.query('in_reply_to_status_id != "NaN"')
#df_ta.query('retweeted_status_id != "NaN"')
df_ta.query('doggo != "None"').head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
9,890240255349198849,NaN,NaN,2017-07-26 15:59:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Cassie. She is a college pup. Studying...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890240255...,14,10,Cassie,doggo,None,None,None
43,884162670584377345,NaN,NaN,2017-07-09 21:29:42 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Meet Yogi. He doesn't have any important dog m...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/884162670...,12,10,Yogi,doggo,None,None,None
99,872967104147763200,NaN,NaN,2017-06-09 00:02:31 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here's a very large dog. He has a date later. ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/872967104...,12,10,None,doggo,None,None,None
108,871515927908634625,NaN,NaN,2017-06-04 23:56:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Napolean. He's a Raggedy East Nicaragu...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/871515927...,12,10,Napolean,doggo,None,None,None
110,871102520638267392,NaN,NaN,2017-06-03 20:33:19 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Never doubt a doggo 14/10 https://t.co/AbBLh2FZCH,NaN,NaN,NaN,https://twitter.com/animalcog/status/871075758...,14,10,None,doggo,None,None,None


In [148]:
df_ta.text[9]

'This is Cassie. She is a college pup. Studying international doggo communication and stick theory. 14/10 so elegant much sophisticate https://t.co/t1bfwz5S2A'

<a id='twitter_api'></a>
#### Request data from the twitter API and load it into a dataframe

Use twitter ID to request retweet count and favourite count.

https://developer.twitter.com/en/docs/labs/tweets-and-users/quick-start/get-tweets

In [290]:
#using .env file and python-dotenv to keep access token safe
#pip install -U python-dotenv

#import os
#from pathlib import Path  # Python 3.6+ only
env_path = Path('.') / '.env'
load_dotenv(dotenv_path=env_path)

consumer_key = os.getenv("TWAPIKEY")
consumer_secret = os.getenv("TWAPISECRETKEY")

#use tweepy to access twitter API with OAuth2

auth = tw.AppAuthHandler(consumer_key, consumer_secret)

#Other option to store passkey safely:
#1. could use a python .config file and the config library to store access token e.g. with wikiart API
#response = requests.get(f'https://www.wikiart.org/en/Api/2/login?accessCode={cfg.twitter['api_key']}&secretCode={cfg.twitter['api_secret_key']')

#2. secure storage of access details with yaml
#import yaml

#with open("config.yml", 'r') as ymlfile:
#    cfg = yaml.safe_load(ymlfile)

#print(cfg[api_creds'access_code'])
#print(cfg[api_creds'secret_code'])

#3.using magic command to access variables in .env
#%env
##Get, set, or list environment variables.

##Usage:

#%env: lists all environment variables/values 
#%env var: get value for var 
#%env var val: set value for var 
#%env var=val: set value for var 
#%env var=$val: set value for var, 
    
##using python expansion if possible



In [57]:
#Access tweets by tweet_id using .get_status() to extract favourites_count, retweet_count and write to csv file
#api.get_status('749075273010798592')._json['retweet_count']

api = tw.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

with open('twitter_retweet_favorite_count.csv', 'a') as file:
    file.write('tweet_id, retweet_count, favorite_count \n')
        
    for i in df_ta['tweet_id']:
        try:
            api_resp = api.get_status(i)
            rt_count=api_resp._json['retweet_count']
            f_count=api_resp._json['favorite_count']
            file.write(f'{i}'+','+f'{rt_count}'+','+f'{f_count}'+'\n')
        except tweepy.TweepError:
            file.write(f'{i}'+',,\n')


Rate limit reached. Sleeping for: 316
Rate limit reached. Sleeping for: 534


In [149]:
df_tapi = pd.read_csv('twitter_retweet_favorite_count.csv')

In [150]:
df_tapi.head()
df_tapi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   tweet_id          2356 non-null   int64  
 1    retweet_count    2331 non-null   float64
 2    favorite_count   2331 non-null   float64
dtypes: float64(2), int64(1)
memory usage: 55.3 KB


In [293]:
## I could also write the json as text one line per tweet into a text file and extract the data later. Will do this do extract more information than we already have
## and possible missing information such as missing extended urls. 

error_dict = {}

api = tw.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

with open('tweet_json.txt', 'a') as file:
        
    for i in df_ta['tweet_id']:
        try:
            api_resp = api.get_status(i)
            json.dump(api_resp._json, file)
            file.write('\n')
           
        except tw.TweepError as e:
            error_dict.append({'error': e, 'tweet_id':i})


Rate limit reached. Sleeping for: 562
Rate limit reached. Sleeping for: 574


In [294]:
print(error_dict)

{'error': TweepError([{'code': 144, 'message': 'No status found with that ID.'}]), 'tweet_id': 680055455951884288}


In [309]:
#Check file length to see how many tweets ids were written to file 
def file_lengthy(fname):
        with open(fname) as f:
                for i, l in enumerate(f):
                        pass
        return i + 1
print("Number of lines in the file: ",file_lengthy("tweet_json.txt"))

Number of lines in the file:  2331


In [151]:
def check_hashtag(tw_json):
    try:
        hashtags = tw_json['entities']['hashtags'][0]['text'] 
    except IndexError:
        hashtags = np.nan
    return hashtags

def check_jpg_url(tw_json):
    try:
        jpg_url = tw_json['entities']['media'][0]['url'] 
    except KeyError:
        jpg_url = np.nan
    return jpg_url

def check_expanded_url(tw_json):
    try:
        expanded_url = tw_json['extended_entities']['media'][0]['expanded_url']
    except KeyError:
        expanded_url = np.nan
    return expanded_url   
                                  
j=0

df_json = pd.DataFrame(columns = ['tweet_id', 'favorite_count', 'retweet_count','hashtags','jpg_url_json', 'expanded_url'])

with open ('tweet_json.txt', 'r') as file:
    for line in file:
        tw_json = json.loads(line)
        j+=1

        hashtags = check_hashtag(tw_json)
        jpg_url = check_jpg_url(tw_json)
        expanded_url = check_expanded_url(tw_json)

        df_json = df_json.append({'tweet_id':tw_json['id'], 'favorite_count':tw_json['favorite_count'], 'retweet_count':tw_json['retweet_count'], \
                                 'hashtags':hashtags,'jpg_url_json':jpg_url, 'expanded_url':expanded_url}, ignore_index=True)       

print(f'Total lines in dataset:{j}')                            
#strip end of text after dataframe has been build.
#df_json.text = df_json['text'].replace(r'\d?\d\/10\s\w.*', '', inplace = True, regex=True)

Total lines in dataset:2331


In [152]:
df_json.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2331 entries, 0 to 2330
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   tweet_id        2331 non-null   object
 1   favorite_count  2331 non-null   object
 2   retweet_count   2331 non-null   object
 3   hashtags        19 non-null     object
 4   jpg_url_json    1816 non-null   object
 5   expanded_url    1816 non-null   object
dtypes: object(6)
memory usage: 109.4+ KB


<a id='requests'></a>
#### Request data from URL and load .tsv file into dataframe 

In [153]:
url='https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
response = requests.get(url)
#response.content

with open('image-predictions.tsv', 'wb') as file:
    file.write(response.content)

df_pre = pd.read_csv('image-predictions.tsv', delimiter='\t')

In [154]:
df_pre.head()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


<a id='assess'></a>
### Part 2 - Assess Data

<a id='visual'></a>
#### Visual assessment: Data overview 

I have four dataframes that all link together with the tweed_id. Two dataframes contain the same data but were extracted differently: df_tapi and df_json. I will use df_json for analysis, as I have extracted extra data in this table that may be useful later on. To analyse the data, the three tables  `df_ta`, `df_json`, and `df_pre`can be merged into one dataset with information about the individual tweets in each row. The outcome of the dog predition algorithm in df_pre are interesting, but not releavnt for my analysis, apart from the most likely dog type prediction for the images send with the tweets.

Some of the columns look redundant, such as expanded twitter url. Need to check if the expansion is always the same. If yes the expanced url can be arrived at using the tweed_id. This information does not have to be stored in a column as this takes up unnecessary space.

Other columns that are not necessary are the probability columns for the second and third best predicitons. I am only interested in the best prediction of the dog type that is actually a dog.

The twitter archive dataframe - df_ta - includes columns that indicate if the tweet was a reply or not (in_reply_to_status_id, in_reply_to_user_id). These columns can be used to filter out any tweets that are not original posts and then the columns can be deleted.

The twitter archive dataframe also includes the text of the tweet, which can be split up and #hashtags, url and rating extracted and stored in separate columns. This column should really only store the text variable. 

<a id='programmatic'></a>
#### Programmatic assessment 

In [319]:
df_ta.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   tweet_id                    2356 non-null   int64  
 1   in_reply_to_status_id       78 non-null     float64
 2   in_reply_to_user_id         78 non-null     float64
 3   timestamp                   2356 non-null   object 
 4   source                      2356 non-null   object 
 5   text                        2356 non-null   object 
 6   retweeted_status_id         181 non-null    float64
 7   retweeted_status_user_id    181 non-null    float64
 8   retweeted_status_timestamp  181 non-null    object 
 9   expanded_urls               2297 non-null   object 
 10  rating_numerator            2356 non-null   int64  
 11  rating_denominator          2356 non-null   int64  
 12  name                        2356 non-null   object 
 13  doggo                       2356 

In [325]:
df_ta.describe()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,retweeted_status_id,retweeted_status_user_id,rating_numerator,rating_denominator
count,2.356000e+03,7.800000e+01,7.800000e+01,1.810000e+02,1.810000e+02,2356.000000,2356.000000
mean,7.427716e+17,7.455079e+17,2.014171e+16,7.720400e+17,1.241698e+16,13.126486,10.455433
std,6.856705e+16,7.582492e+16,1.252797e+17,6.236928e+16,9.599254e+16,45.876648,6.745237
min,6.660209e+17,6.658147e+17,1.185634e+07,6.661041e+17,7.832140e+05,0.000000,0.000000
25%,6.783989e+17,6.757419e+17,3.086374e+08,7.186315e+17,4.196984e+09,10.000000,10.000000
50%,7.196279e+17,7.038708e+17,4.196984e+09,7.804657e+17,4.196984e+09,11.000000,10.000000
75%,7.993373e+17,8.257804e+17,4.196984e+09,8.203146e+17,4.196984e+09,12.000000,10.000000
max,8.924206e+17,8.862664e+17,8.405479e+17,8.874740e+17,7.874618e+17,1776.000000,170.000000


In [320]:
df_tapi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   tweet_id          2356 non-null   int64  
 1    retweet_count    2331 non-null   float64
 2    favorite_count   2331 non-null   float64
dtypes: float64(2), int64(1)
memory usage: 55.3 KB


In [326]:
df_tapi.describe()

,tweet_id,retweet_count,favorite_count
count,2.356000e+03,2331.000000,2331.000000
mean,7.427716e+17,2647.039468,7426.154869
std,6.856705e+16,4476.566497,11532.166948
min,6.660209e+17,1.000000,0.000000
25%,6.783989e+17,536.500000,1291.000000
50%,7.196279e+17,1237.000000,3226.000000
75%,7.993373e+17,3072.000000,9092.000000
max,8.924206e+17,76063.000000,153368.000000


In [321]:
df_json.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2331 entries, 0 to 2330
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   twitter_id      2331 non-null   object
 1   favorite_count  2331 non-null   object
 2   retweet_count   2331 non-null   object
 3   hashtags        19 non-null     object
 4   jpg_url         1816 non-null   object
 5   expanded_url    2331 non-null   object
dtypes: object(6)
memory usage: 109.4+ KB


In [328]:
df_json.describe()

,twitter_id,favorite_count,retweet_count,hashtags,jpg_url,expanded_url
count,2331,2331,2331,19,1816,2331
unique,2331,1984,1673,16,1757,1757
top,667495797102141441,0,799,BarkWeek,https://t.co/MiMKtsLN6k,https://twitter.com/dog_rates/status/872620804844003328/photo/1
freq,1,163,5,3,2,15


In [334]:
for col in df_json:
    print(f'{col}: {type(col[0])}')

twitter_id: <class 'str'>
favorite_count: <class 'str'>
retweet_count: <class 'str'>
hashtags: <class 'str'>
jpg_url: <class 'str'>
expanded_url: <class 'str'>


In [372]:
df_pre.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   tweet_id  2075 non-null   int64  
 1   jpg_url   2075 non-null   object 
 2   img_num   2075 non-null   int64  
 3   p1        2075 non-null   object 
 4   p1_conf   2075 non-null   float64
 5   p1_dog    2075 non-null   bool   
 6   p2        2075 non-null   object 
 7   p2_conf   2075 non-null   float64
 8   p2_dog    2075 non-null   bool   
 9   p3        2075 non-null   object 
 10  p3_conf   2075 non-null   float64
 11  p3_dog    2075 non-null   bool   
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


<a id='structure'></a>
#### Data structure: Tidyness issues

**Issue:** in_reply_to_status, in_reply_to_user_id. Use columns to drop rows that are replies and not original ids and delete both columns.

**Issue:** Find best prediction for each dog that is actually a dog and store it in a column. Merge this and the image_url column with df_ta dataframe. All other columns are redundant.

**Issue:** Merge df_json columns with df_ta dataframe on the tweed_id column. Delete df_json dataframe

**Issue:** Doggo, Floofer, Pupper, Puppo columns need to be melted into one column - dog_age_category: which contains the category which is correct for the dog.

**Issue:** More then one variable stored in the df_ta.text column. Removes #tags, urls and ratings from text as these can be store or are stored in other columns. Two different urls are saved for the tweet. Once in the df_ta.text column and once in df_pr.jpg_id. The rating are stored in two separate columns: df_ta.denominator and df_ta.numerator. Endresult: the text col only contains the text.
    

In [884]:
df_ta[df_ta.text.str.contains('#')].text.count()

27

In [885]:
df_ta[df_ta.text.str.contains('https:')].text.count()

2261

In [886]:
#df_ta.text[12]
df_ta.query('tweet_id == 892420643555336193').text[0] 

"This is Phineas. He's a mystical boy. Only ever appears in the hole of a donut. 13/10 https://t.co/MgUWQ76dJU"

In [890]:
pd.options.display.max_colwidth=100
df_ta.query('tweet_id == 871102520638267392').expanded_urls ##the /photo/1 does not work. This url returns the tweet itself.

1910    https://twitter.com/animalcog/status/871075758080503809
Name: expanded_urls, dtype: object

<a id="quality"></a>
#### Data quality: 

**Issue:** wrong datatypes:

             df_ta: timestamp, source (category), retweeted_status_id, retweeted_status_user_id, retweeted_status_timestamp
         
             df_json: tweet_id, retweet_count, favorite_count

**Issue:** suspicious rating_numerators (< 8 and > 15)

In [107]:
df_ta.rating_numerator.value_counts()

12      558
11      464
10      461
13      351
9       158
8       102
7        55
14       54
5        37
6        32
3        19
4        17
1         9
2         9
420       2
0         2
15        2
75        2
80        1
20        1
24        1
26        1
44        1
50        1
60        1
165       1
84        1
88        1
144       1
182       1
143       1
666       1
960       1
1776      1
17        1
27        1
45        1
99        1
121       1
204       1
Name: rating_numerator, dtype: int64

**Issue:** suspicious denominator values. Especially 110, 120, etc. which most likely have been read in with a zero too much.

In [108]:
df_ta.rating_denominator.value_counts()

10     2333
11        3
50        3
80        2
20        2
2         1
16        1
40        1
70        1
15        1
90        1
110       1
120       1
130       1
150       1
170       1
7         1
0         1
Name: rating_denominator, dtype: int64

**Issue:** Source should be a categorical column as there are only 4 options: 
<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>     2221
<a href="http://vine.co" rel="nofollow">Vine - Make a Scene</a>                          91
<a href="http://twitter.com" rel="nofollow">Twitter Web Client</a>                       33
<a href="https://about.twitter.com/products/tweetdeck" rel="nofollow">TweetDeck</a>      11

Should be stored as: 'Twitter for iPhone', 'Vine - Make a Scene', 'Twitter Web Client', 'TweetDeck'. The href for these categories can be stored elsewhere.

**Issue**: Image url is stored twice. End of text and in df_json?

**Issue:** Missing value in expanded_url column. This data is stored in the json file and can be extracted to be added. Maybe add more interesting data that is missing.

**Issue:** Check dog names count_values. Wrong dog names (a, the, one, quite, mad, not, 0, life, space, this, by, officially, old, his, such, inacceptable, my, all, incredibly. See if I can restore these names from the text.

In [182]:
df_ta.name.value_counts()

None              745
a                  55
Charlie            12
Cooper             11
Lucy               11
Oliver             11
Penny              10
Tucker             10
Lola               10
Winston             9
Bo                  9
the                 8
Sadie               8
Bailey              7
Daisy               7
Toby                7
Buddy               7
an                  7
Koda                6
Leo                 6
Rusty               6
Bella               6
Jack                6
Scout               6
Stanley             6
Milo                6
Dave                6
Oscar               6
Jax                 6
Alfie               5
very                5
Sunny               5
Phil                5
Oakley              5
Sammy               5
Louis               5
Larry               5
Finn                5
Bentley             5
George              5
Gus                 5
Chester             5
Gerald              4
Clark               4
Shadow              4
Brody     

**Issue:** The df_ta.text contains sometimes two dogs and only one name is stored but both dog categories. Or it refers to two categories of dogs, e.g. doggo 1, pupper 0. Where both categories are set to true. 

In [184]:
df_ta.query('doggo == "doggo" and pupper =="pupper"')

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
460,817777686764523521,NaN,NaN,2017-01-07 16:59:28 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","This is Dido. She's playing the lead role in ""Pupper Stops to Catch Snow Before Resuming Shadow Box with Dried Apple."" 13/10 (IG: didodoggo) https://t.co/m7isZrOBX7",NaN,NaN,NaN,https://twitter.com/dog_rates/status/817777686764523521/video/1,13,10,Dido,doggo,None,pupper,None
531,808106460588765185,NaN,NaN,2016-12-12 00:29:28 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",Here we have Burke (pupper) and Dexter (doggo). Pupper wants to be exactly like doggo. Both 12/10 would pet at same time https://t.co/ANBpEYHaho,NaN,NaN,NaN,https://twitter.com/dog_rates/status/808106460588765185/photo/1,12,10,None,doggo,None,pupper,None
565,802265048156610565,7.331095e+17,4.196984e+09,2016-11-25 21:37:47 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","Like doggo, like pupper version 2. Both 11/10 https://t.co/9IxWAXFqze",NaN,NaN,NaN,https://twitter.com/dog_rates/status/802265048156610565/photo/1,11,10,None,doggo,None,pupper,None
575,801115127852503040,NaN,NaN,2016-11-22 17:28:25 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Bones. He's being haunted by another doggo of roughly the same size. 12/10 deep breaths pupper everything's fine https://t.co/55Dqe0SJNj,NaN,NaN,NaN,"https://twitter.com/dog_rates/status/801115127852503040/photo/1,https://twitter.com/dog_rates/status/801115127852503040/photo/1",12,10,Bones,doggo,None,pupper,None
705,785639753186217984,NaN,NaN,2016-10-11 00:34:48 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Pinot. He's a sophisticated doggo. You can tell by the hat. Also pointier than your average pupper. Still 10/10 would pet cautiously https://t.co/f2wmLZTPHd,NaN,NaN,NaN,"https://twitter.com/dog_rates/status/785639753186217984/photo/1,https://twitter.com/dog_rates/status/785639753186217984/photo/1",10,10,Pinot,doggo,None,pupper,None
733,781308096455073793,NaN,NaN,2016-09-29 01:42:20 +0000,"<a href=""http://vine.co"" rel=""nofollow"">Vine - Make a Scene</a>","Pupper butt 1, Doggo 0. Both 12/10 https://t.co/WQvcPEpH2u",NaN,NaN,NaN,https://vine.co/v/5rgu2Law2ut,12,10,None,doggo,None,pupper,None
778,775898661951791106,NaN,NaN,2016-09-14 03:27:11 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","RT @dog_rates: Like father (doggo), like son (pupper). Both 12/10 https://t.co/pG2inLaOda",7.331095e+17,4.196984e+09,2016-05-19 01:38:16 +0000,"https://twitter.com/dog_rates/status/733109485275860992/photo/1,https://twitter.com/dog_rates/status/733109485275860992/photo/1",12,10,None,doggo,None,pupper,None
822,770093767776997377,NaN,NaN,2016-08-29 03:00:36 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",RT @dog_rates: This is just downright precious af. 12/10 for both pupper and doggo https://t.co/o5J479bZUC,7.410673e+17,4.196984e+09,2016-06-10 00:39:48 +0000,"https://twitter.com/dog_rates/status/741067306818797568/photo/1,https://twitter.com/dog_rates/status/741067306818797568/photo/1",12,10,just,doggo,None,pupper,None
889,759793422261743616,NaN,NaN,2016-07-31 16:50:42 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","Meet Maggie &amp; Lila. Maggie is the doggo, Lila is the pupper. They are sisters. Both 12/10 would pet at the same time https://t.co/MYwR4DQKll",NaN,NaN,NaN,"https://twitter.com/dog_rates/status/759793422261743616/photo/1,https://twitter.com/dog_rates/status/759793422261743616/photo/1",12,10,Maggie,doggo,Non

**Issue:** Predicted dog names are written inconsistently. Should be without - and capitalised.

<a id='clean'></a>
### Part III - Clean data and create twitter_archive_master.csv file

<a id='tidy'></a> 
### Data Tidiness issues

<a id="def1"> </a>
#### Define issue: In_reply_to_status, in_reply_to_user_id

The columns `in_reply_to_status, in_reply_to_user_id` store replies to original posts, that I do not want to investigate. Drop rows with data from replies, i.e. where in_reply_to_status_id is not Null and then drop both columns `in_reply_to_status, in_reply_to_user_id `from the table

#### Code issue:

In [155]:
df_ta = df_ta.query('in_reply_to_status_id == "NaN"')
df_ta.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2278 entries, 0 to 2355
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   tweet_id                    2278 non-null   int64  
 1   in_reply_to_status_id       0 non-null      float64
 2   in_reply_to_user_id         0 non-null      float64
 3   timestamp                   2278 non-null   object 
 4   source                      2278 non-null   object 
 5   text                        2278 non-null   object 
 6   retweeted_status_id         181 non-null    float64
 7   retweeted_status_user_id    181 non-null    float64
 8   retweeted_status_timestamp  181 non-null    object 
 9   expanded_urls               2274 non-null   object 
 10  rating_numerator            2278 non-null   int64  
 11  rating_denominator          2278 non-null   int64  
 12  name                        2278 non-null   object 
 13  doggo                       2278 

In [156]:
df_ta.drop(['in_reply_to_status_id', 'in_reply_to_user_id'], axis=1, inplace=True)

#### Test issue:

In [157]:
df_ta.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2278 entries, 0 to 2355
Data columns (total 15 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   tweet_id                    2278 non-null   int64  
 1   timestamp                   2278 non-null   object 
 2   source                      2278 non-null   object 
 3   text                        2278 non-null   object 
 4   retweeted_status_id         181 non-null    float64
 5   retweeted_status_user_id    181 non-null    float64
 6   retweeted_status_timestamp  181 non-null    object 
 7   expanded_urls               2274 non-null   object 
 8   rating_numerator            2278 non-null   int64  
 9   rating_denominator          2278 non-null   int64  
 10  name                        2278 non-null   object 
 11  doggo                       2278 non-null   object 
 12  floofer                     2278 non-null   object 
 13  pupper                      2278 

<a id=def2> </a>
#### Define Issues: Find best dog type prediction and merge this, the img_num and image_url column with df_ta dataframe. Inconsistent dog names.

In [158]:
df_pre.head()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


The `df_pre`table contains three predictions of what type of dog is likely to be shown on the image. Delete all data where all three predictions are not a dog. Insert column final_prediction and set to dog 1. Then use a mask to set final prediction to dog 2 if dog 1 is False. And finally set it to dog 3 if dog 2 is False using .mask(). While I am working on this table, I have also changed the dog names to a consistend format.

<a id=code2> </a>
#### Code Issue 2 + 9: 

In [159]:
df_pre = df_pre.query('p1_dog == True or p2_dog == True or p3_dog == True')
df_pre.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1751 entries, 0 to 2073
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   tweet_id  1751 non-null   int64  
 1   jpg_url   1751 non-null   object 
 2   img_num   1751 non-null   int64  
 3   p1        1751 non-null   object 
 4   p1_conf   1751 non-null   float64
 5   p1_dog    1751 non-null   bool   
 6   p2        1751 non-null   object 
 7   p2_conf   1751 non-null   float64
 8   p2_dog    1751 non-null   bool   
 9   p3        1751 non-null   object 
 10  p3_conf   1751 non-null   float64
 11  p3_dog    1751 non-null   bool   
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 141.9+ KB


In [160]:
## find the best prediction
df_pre['dog_type'] = df_pre['p1']
df_pre['dog_type_flag'] = df_pre.p1_dog
df_pre['dog_type'] = df_pre['dog_type'].mask(df_pre.dog_type_flag == False, df_pre['p2'])
df_pre['dog_type_flag'] = df_pre['dog_type_flag'].mask(df_pre.dog_type_flag == False, df_pre['p2_dog'])
df_pre['dog_type'] = df_pre['dog_type'].mask(df_pre.dog_type_flag == False, df_pre['p3'])
df_pre['dog_type_flag'] = df_pre['dog_type_flag'].mask(df_pre.dog_type_flag == False, df_pre['p3_dog'])

df_pre.dog_type_flag.value_counts()
#df_pre.dog_type.value_counts()

True    1751
Name: dog_type_flag, dtype: int64

In [161]:
## Change dog name format
df_pre['dog_type'] = df_pre['dog_type'].str.replace('_', ' ')
df_pre['dog_type'] = df_pre['dog_type'].apply(lambda x: x.title())
df_pre_short = df_pre[['tweet_id', 'jpg_url', 'img_num', 'dog_type']]

df_pre_short.head(50)

,tweet_id,jpg_url,img_num,dog_type
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh Springer Spaniel
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,Redbone
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German Shepherd
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian Ridgeback
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,Miniature Pinscher
5,666050758794694657,https://pbs.twimg.com/media/CT5Jof1WUAEuVxN.jpg,1,Bernese Mountain Dog
7,666055525042405380,https://pbs.twimg.com/media/CT5N9tpXIAAifs1.jpg,1,Chow
8,666057090499244032,https://pbs.twimg.com/media/CT5PY90WoAAQGLo.jpg,1,Golden Retriever
9,666058600524156928,https://pbs.twimg.com/media/CT5Qw94XAAA_2dP.jpg,1,Miniature Poodle
10,666063827256086533,https://pbs.twimg.com/media/CT5Vg_wXIAAXfnj.jpg,1,Golden Retriever


In [162]:
df_ta = pd.merge(df_ta, df_pre_short, how = 'left', on='tweet_id')

<a id=test2> </a>
#### Test Issue 2 + 9: 

In [163]:
df_ta.head()

,tweet_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo,jpg_url,img_num,dog_type
0,892420643555336193,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None,NaN,NaN,NaN
1,892177421306343426,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None,https://pbs.twimg.com/media/DGGmoV4XsAAUL6n.jpg,1.0,Chihuahua
2,891815181378084864,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None,https://pbs.twimg.com/media/DGBdLU1WsAANxJ9.jpg,1.0,Chihuahua
3,891689557279858688,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None,https://pbs.twimg.com/media/DF_q7IAWsAEuuN8.jpg,1.0,Labrador Retriever
4,891327558926688256,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None,https://pbs.twimg.com/media/DF6hr6BUMAAzZgT.jpg,2.0,Basset


<a id='def3'></a>
#### Define Issue 3: Merge df_json columns with df_ta dataframe on the tweed_id column. Delete df_json dataframe

The table df_json can not be merged yet, as the datatypes of the twitter_id is string. Need to convert this to int64 so that merging is possible. While I am already changing datatypes, I am also changing favorite_count and retweet_count to int64.

<a id='clean3'></a>
#### Clean Issue 3:

In [164]:
df_json.tweet_id = df_json.tweet_id.astype('int64')
df_json.favorite_count = df_json.favorite_count.astype('int64')
df_json.retweet_count = df_json.retweet_count.astype('int64')

In [165]:
df_ta = pd.merge(df_ta, df_json, how='left', on='tweet_id')

In [166]:
del df_json

<a id='test3'></a>
#### Test Issue 3:

In [167]:
#df_json.info()
df_ta.head()

,tweet_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,...,pupper,puppo,jpg_url,img_num,dog_type,favorite_count,retweet_count,hashtags,jpg_url_json,expanded_url
0,892420643555336193,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,...,None,None,NaN,NaN,NaN,35568.0,7523.0,NaN,https://t.co/MgUWQ76dJU,https://twitter.com/dog_rates/status/892420643...
1,892177421306343426,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,...,None,None,https://pbs.twimg.com/media/DGGmoV4XsAAUL6n.jpg,1.0,Chihuahua,NaN,NaN,NaN,NaN,NaN
2,891815181378084864,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,...,None,None,https://pbs.twimg.com/media/DGBdLU1WsAANxJ9.jpg,1.0,Chihuahua,23141.0,3701.0,NaN,NaN,NaN
3,891689557279858688,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,...,None,None,https://pbs.twimg.com/media/DF_q7IAWsAEuuN8.jpg,1.0,Labrador Retriever,38857.0,7711.0,NaN,https://t.co/tD36da7qLQ,https://twitter.com/dog_rates/status/891689557...
4,891327558926688256,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,...,None,None,https://pbs.twimg.com/media/DF6hr6BUMAAzZgT.jpg,2.0,Basset,37132.0,8308.0,NaN,NaN,NaN


<a id='def4'></a>
#### Define issue 4 + issue 9: Doggo, Floofer, Pupper, Puppo columns need to be melted into one column - dog_age_category


Doggo, Floofer, Pupper, Puppo are categories of dogs, that can be melted into one column - dog_age_category using melt(). The value of the dog_age_category is 'Fluffer', 'Pupper' etc. Need to be careful as some comments talk about more than one dog and more than one category can then be set to True. Or sometimes two age categories are referenced but only one applies to the dog. Therefore I will first delete all rows that have two dog age categories set to true, as I don't know which one is the correct one for the main dog, listed in dog name.

<a id='code4'></a>
#### Code issue 4: 

In [168]:
df_ta.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2278 entries, 0 to 2277
Data columns (total 23 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   tweet_id                    2278 non-null   int64  
 1   timestamp                   2278 non-null   object 
 2   source                      2278 non-null   object 
 3   text                        2278 non-null   object 
 4   retweeted_status_id         181 non-null    float64
 5   retweeted_status_user_id    181 non-null    float64
 6   retweeted_status_timestamp  181 non-null    object 
 7   expanded_urls               2274 non-null   object 
 8   rating_numerator            2278 non-null   int64  
 9   rating_denominator          2278 non-null   int64  
 10  name                        2278 non-null   object 
 11  doggo                       2278 non-null   object 
 12  floofer                     2278 non-null   object 
 13  pupper                      2278 

In [169]:
##Extract all data with dog_category values into a separate table to melt()
df_ta_dog_age = df_ta.query('doggo == "doggo" or floofer == "floofer" or pupper =="pupper" or puppo =="puppo" ')
df_ta_dog_age = df_ta_dog_age.melt(id_vars=['tweet_id', 'timestamp', 'source', 'text', 'retweeted_status_id', 'retweeted_status_user_id','retweeted_status_timestamp', 'expanded_urls', 'rating_numerator', 'rating_denominator', \
            'name', 'jpg_url', 'img_num', 'dog_type', 'favorite_count', 'retweet_count', 'hashtags', 'jpg_url_json', 'expanded_url'], var_name='dog_age_category', value_name='dog_age')


In [170]:
##Delete extra rows generated by melt to keep only the ones that have a dog_age value
df_ta_dog_age = df_ta_dog_age.query('dog_age != "None"')
df_ta_dog_age
##delete dog_age column as dog_age_category has all the information that we need
df_ta_dog_age.drop(columns = 'dog_age', inplace = True)

In [171]:
df_ta_dog_age.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 385 entries, 0 to 1313
Data columns (total 20 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   tweet_id                    385 non-null    int64  
 1   timestamp                   385 non-null    object 
 2   source                      385 non-null    object 
 3   text                        385 non-null    object 
 4   retweeted_status_id         38 non-null     float64
 5   retweeted_status_user_id    38 non-null     float64
 6   retweeted_status_timestamp  38 non-null     object 
 7   expanded_urls               384 non-null    object 
 8   rating_numerator            385 non-null    int64  
 9   rating_denominator          385 non-null    int64  
 10  name                        385 non-null    object 
 11  jpg_url                     278 non-null    object 
 12  img_num                     278 non-null    float64
 13  dog_type                    278 no

In [172]:
##drop data with dog type information in the df_at table and add data from df_ta_dog_age table to original table.

In [173]:
df_ta = df_ta.query('doggo != "doggo" and floofer != "floofer" and pupper !="pupper" and puppo != "puppo"')
#df_ta.puppo.value_counts()
df_ta = df_ta.drop(columns=['doggo', 'pupper', 'puppo', 'floofer'])

In [174]:
##find duplicates for dogs that have two categories in df_ta_dog_age: (Reset index to be able to identify the duplicate columns that need to be dropped)
df_ta_dog_age.reset_index(inplace = True) 
tweet_ids_with_two_dog_cats = df_ta_dog_age[df_ta_dog_age.tweet_id.duplicated()].sort_values('tweet_id').tweet_id

In [175]:
#df_ta_dog_age.query('tweet_id in @tweet_ids_with_two_dog_cats').sort_values('tweet_id') - 13 Duplicates
df_ta_dog_age.query('tweet_id in @tweet_ids_with_two_dog_cats').sort_values('tweet_id').index

Int64Index([ 87, 196,  83, 189,  75, 174,  67, 169,  62, 164,  57, 159,  54,
            152,  50, 151,  44, 140,  42, 137, 130,  37,   8,  94,   7, 362],
           dtype='int64')

In [176]:
##Rows to delete as they are the wrong category for this dog or because there are two dogs and two categories, so one is being deleted:

df_ta_dog_age.drop(index=[196,83,75,67,62,57,54,151,96,42,37,8,7], inplace=True)

#Rows to update - for all the tweets with two dogs, the dog_age_category is being set to "":
df_ta_dog_age.loc[87,'dog_age_category'] = ''
df_ta_dog_age.loc[189,'dog_age_category'] = ''
df_ta_dog_age.loc[174,'dog_age_category'] = '' # delete this as there is no dog on this?
df_ta_dog_age.loc[169,'dog_age_category'] = ''
df_ta_dog_age.loc[164,'dog_age_category'] = ''
df_ta_dog_age.loc[159,'dog_age_category'] = ''
df_ta_dog_age.loc[137,'dog_age_category'] = ''


In [177]:
df_ta['dog_age_category']=np.nan
df_ta.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1906 entries, 0 to 2277
Data columns (total 20 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   tweet_id                    1906 non-null   int64  
 1   timestamp                   1906 non-null   object 
 2   source                      1906 non-null   object 
 3   text                        1906 non-null   object 
 4   retweeted_status_id         145 non-null    float64
 5   retweeted_status_user_id    145 non-null    float64
 6   retweeted_status_timestamp  145 non-null    object 
 7   expanded_urls               1903 non-null   object 
 8   rating_numerator            1906 non-null   int64  
 9   rating_denominator          1906 non-null   int64  
 10  name                        1906 non-null   object 
 11  jpg_url                     1464 non-null   object 
 12  img_num                     1464 non-null   float64
 13  dog_type                    1464 

In [178]:
 ## create new column on df_ta: `dog_age_category` and merge data from df_ta_dog_age to df_ta using outerjoin to add the melted data back to the original dataframe
df_ta = df_ta.append(df_ta_dog_age, ignore_index=True)
df_ta = df_ta.drop(columns='index')

<a id='test4'></a>
#### Test issue 4: 

In [179]:
df_ta_dog_age.info()
df_ta.query('tweet_id in (@tweet_ids_with_two_dog_cats ) ')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 372 entries, 0 to 384
Data columns (total 21 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   index                       372 non-null    int64  
 1   tweet_id                    372 non-null    int64  
 2   timestamp                   372 non-null    object 
 3   source                      372 non-null    object 
 4   text                        372 non-null    object 
 5   retweeted_status_id         36 non-null     float64
 6   retweeted_status_user_id    36 non-null     float64
 7   retweeted_status_timestamp  36 non-null     object 
 8   expanded_urls               371 non-null    object 
 9   rating_numerator            372 non-null    int64  
 10  rating_denominator          372 non-null    int64  
 11  name                        372 non-null    object 
 12  jpg_url                     267 non-null    object 
 13  img_num                     267 non

,tweet_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,jpg_url,img_num,dog_type,favorite_count,retweet_count,hashtags,jpg_url_json,expanded_url,dog_age_category
1946,801115127852503040,2016-11-22 17:28:25 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Bones. He's being haunted by another d...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/801115127...,12,10,Bones,https://pbs.twimg.com/media/Cx4h7zHUsAAqaJd.jpg,1.0,Dalmatian,8091.0,2108.0,NaN,NaN,NaN,doggo
1952,785639753186217984,2016-10-11 00:34:48 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Pinot. He's a sophisticated doggo. You...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/785639753...,10,10,Pinot,NaN,NaN,NaN,7858.0,2241.0,NaN,NaN,NaN,doggo
1983,733109485275860992,2016-05-19 01:38:16 +0000,"<a href=""http://twitter.com/download/iphone"" r...","Like father (doggo), like son (pupper). Both 1...",NaN,NaN,NaN,https://twitter.com/dog_rates/status/733109485...,12,10,None,https://pbs.twimg.com/media/CiyHLocU4AI2pJu.jpg,1.0,Golden Retriever,49063.0,16409.0,NaN,https://t.co/pG2inLaOda,https://twitter.com/dog_rates/status/733109485...,
1990,854010172552949760,2017-04-17 16:34:26 +0000,"<a href=""http://twitter.com/download/iphone"" r...","At first I thought this was a shy doggo, but i...",NaN,NaN,NaN,https://twitter.com/dog_rates/status/854010172...,11,10,None,https://pbs.twimg.com/media/C9oNt91WAAAFSLS.jpg,1.0,English Springer,15606.0,2995.0,NaN,NaN,NaN,floofer
2025,817777686764523521,2017-01-07 16:59:28 +0000,"<a href=""http://twitter.com/download/iphone"" r...","This is Dido. She's playing the lead role in ""...",NaN,NaN,NaN,https://twitter.com/dog_rates/status/817777686...,13,10,Dido,https://pbs.twimg.com/ext_tw_video_thumb/81777...,1.0,Curly-Coated Retriever,NaN,NaN,NaN,NaN,NaN,pupper
2032,808106460588765185,2016-12-12 00:29:28 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here we have Burke (pupper) and Dexter (doggo)...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/808106460...,12,10,None,https://pbs.twimg.com/media/Czb4iFRXgAIUMiN.jpg,1.0,Golden Retriever,NaN,NaN,NaN,NaN,NaN,
2035,801115127852503040,2016-11-22 17:28:25 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Bones. He's being haunted by another d...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/801115127...,12,10,Bones,https://pbs.twimg.com/media/Cx4h7zHUsAAqaJd.jpg,1.0,Dalmatian,8091.0,2108.0,NaN,NaN,NaN,pupper
2046,781308096455073793,2016-09-29 01:42:20 +0000,"<a href=""http://vine.co"" rel=""nofollow"">Vine -...","Pupper butt 1, Doggo 0. Both 12/10 https://t.c...",NaN,NaN,NaN,https://vine.co/v/5rgu2Law2ut,12,10,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,pupper
2053,775898661951791106,2016-09-14 03:27:11 +0000,"<a href=""http://twitter.com/download/iphone"" r...","RT @dog_rates: Like father (doggo), like son (...",7.331095e+17,4.196984e+09,2016-05-19 01:38:16 +0000,https://twitter.com/dog_rates/status/733109485...,12,10,None,https://pbs.twimg.com/media/CiyHLocU4AI2pJu.jpg,1.0,Golden Retriever,0.0,16409.0,NaN,https://t.co/pG2inLaOda,https://twitter.com/dog_rates/status/733109485...,
2058,770093767776997377,2016-08-29 03:00:36 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: This is just downright precious...,7.410673e+17,4.196984e+09,2016-06-10 00:39:48 +0000,https://twitter.com/dog_rates/status/741067306...,12,10,just,https://pbs.twimg.com/media/CkjMx99UoAM2B1a.jpg,1.0,Golden Retriever,0.0,3024.0,NaN,https://t.co/o5J479bZUC,https://twitter.com/dog_rates/status/741067306...,


In [180]:
df_ta.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2278 entries, 0 to 2277
Data columns (total 20 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   tweet_id                    2278 non-null   int64  
 1   timestamp                   2278 non-null   object 
 2   source                      2278 non-null   object 
 3   text                        2278 non-null   object 
 4   retweeted_status_id         181 non-null    float64
 5   retweeted_status_user_id    181 non-null    float64
 6   retweeted_status_timestamp  181 non-null    object 
 7   expanded_urls               2274 non-null   object 
 8   rating_numerator            2278 non-null   int64  
 9   rating_denominator          2278 non-null   int64  
 10  name                        2278 non-null   object 
 11  jpg_url                     1731 non-null   object 
 12  img_num                     1731 non-null   float64
 13  dog_type                    1731 

<a id='quality'></a> 
### Data Quality issues

  <a id="def1"> </a>
#### Define issue: Wrong data types and extract source as categorical data

             timestamp - timestamp
             source  - category, 
             retweeted_status_id  - int64,
             retweeted_status_user_id - int64, 
             retweeted_status_timestamp - timestamp, 
             img_num  - int64
             favorite_count - int_64
             retweet_count - int_64
             dog_age_category - category
             

  <a id="codeq1"> </a>
#### Code issue:

In [201]:
df_ta.timestamp = pd.to_datetime(df_ta.timestamp)
df_ta.retweeted_status_id = df_ta.retweeted_status_id.astype('Int64')
df_ta.retweeted_status_user_id = df_ta.retweeted_status_user_id.astype('Int64')
df_ta.retweeted_status_timestamp = pd.to_datetime(df_ta.retweeted_status_timestamp)
df_ta.img_num = df_ta.img_num.astype('Int64')
df_ta.favorite_count = df_ta.favorite_count.astype('Int64')
df_ta.retweet_count = df_ta.retweet_count.astype('Int64')
df_ta.dog_age_category = df_ta.dog_age_category.astype('category')
df_ta.rating_numerator = df_ta.rating_numerator.astype('float')
df_ta.rating_denominator = df_ta.rating_denominator.astype('float')

In [186]:
#extract source tag from a href.
#<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a> is going to be `Twitter for iPhone``
#<a href="http://vine.co" rel="nofollow">Vine - Make a Scene</a>  is going to be `Vine - Make a Scene`
#<a href="http://twitter.com" rel="nofollow">Twitter Web Client</a>  is going to be `Twitter Web Client`
#<a href="https://about.twitter.com/products/tweetdeck" rel="nofollow">TweetDeck</a> is going to be `TweetDeck`

df_ta.source = df_ta.source.str.extract(r'((?:<a href=".*">)(.*)<.)', expand=True)[1]
df_ta.source.value_counts()
df_ta.source = df_ta.source.astype('category')

  <a id="testq1"> </a>
#### Test issue:

In [202]:
df_ta.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2278 entries, 0 to 2277
Data columns (total 20 columns):
 #   Column                      Non-Null Count  Dtype              
---  ------                      --------------  -----              
 0   tweet_id                    2278 non-null   int64              
 1   timestamp                   2278 non-null   datetime64[ns, UTC]
 2   source                      2278 non-null   category           
 3   text                        2278 non-null   object             
 4   retweeted_status_id         181 non-null    Int64              
 5   retweeted_status_user_id    181 non-null    Int64              
 6   retweeted_status_timestamp  181 non-null    datetime64[ns, UTC]
 7   expanded_urls               2274 non-null   object             
 8   rating_numerator            2278 non-null   float64            
 9   rating_denominator          2278 non-null   float64            
 10  name                        2278 non-null   object          

<a id='defq2'></a>
#### Defining issue: Suspicious numerator and denominator rating values

Denominators of the ratings should always be 10, as the rating is out of 10. For Numerator values there are some very high and low values. For some it looks like a 0 has been added on the end by mistake. Check original text to see what the rating was and trim excess rating or extract ratings again and store as ratings_numerator and ratings_denominator

<a id='codeq2'></a>
#### Code issue:

In [204]:
df_ta.rating_numerator.value_counts().index.sort_values()
df_ta[['text', 'rating_numerator']].query('rating_numerator > 15')

,text,rating_numerator
341,The floofs have been released I repeat the floofs have been released. 84/70 https://t.co/NIYC820tmd,84.0
402,Meet Sam. She smiles 24/7 &amp; secretly aspires to be a reindeer. \nKeep Sam smiling by clicking and sharing this link:\nhttps://t.co/98tB8y7y7t https://t.co/LouL5vdvxx,24.0
549,"This is Logan, the Chow who lived. He solemnly swears he's up to lots of good. H*ckin magical af 9.75/10 https://t.co/yBO5wuqaPS",75.0
716,Why does this never happen at my front door... 165/150 https://t.co/HmwrdfEfUE,165.0
772,This is Atticus. He's quite simply America af. 1776/10 https://t.co/GRXwMxLBkh,1776.0
875,Say hello to this unbelievably well behaved squad of doggos. 204/170 would try to pet all at once https://t.co/yGQI3He3xv,204.0
939,This is Bluebert. He just saw that both #FinalFur match ups are split 50/50. Amazed af. 11/10 https://t.co/Kky1DPG4iq,50.0
964,Happy Saturday here's 9 puppers on a bench. 99/90 good work everybody https://t.co/mpvaVxKmc1,99.0
987,Here's a brigade of puppers. All look very prepared for whatever happens next. 80/80 https://t.co/0eb7R1Om12,80.0
1006,"From left to right:\nCletus, Jerome, Alejandro, Burp, &amp; Titson\nNone know where camera is. 45/50 would hug all at once https://t.co/sedre1ivTK",45.0


In [228]:
#leave 341, 716, 772, 875, 964, 987, 1006, etc.  -- often more than one dog and then they are rated out of 50 if there are five dogs etc.
df_ta.loc[276,'rating_numerator'] = 9.75
df_ta.loc[402,'rating_numerator'] = np.nan
df_ta.loc[549,'rating_numerator'] = 9.75
df_ta.loc[939,'rating_numerator'] = 11
df_ta.loc[1340,'rating_numerator'] = 11.26
df_ta.loc[2049,'rating_numerator'] = 11.27
df_ta.loc[909,'rating_numerator'] = 13
df_ta.loc[617,'rating_numerator'] = 14
df_ta.loc[837, 'rating_numerator'] = 14
df_ta.loc[1885, 'rating_numerator'] = 9
df_ta.loc[1300, 'rating_numerator'] = 10

In [229]:
df_ta.rating_denominator.value_counts().index.sort_values()
df_ta[['text', 'rating_denominator']].query('rating_denominator > 10')

,text,rating_denominator
341,The floofs have been released I repeat the floofs have been released. 84/70 https://t.co/NIYC820tmd,70.0
716,Why does this never happen at my front door... 165/150 https://t.co/HmwrdfEfUE,150.0
875,Say hello to this unbelievably well behaved squad of doggos. 204/170 would try to pet all at once https://t.co/yGQI3He3xv,170.0
964,Happy Saturday here's 9 puppers on a bench. 99/90 good work everybody https://t.co/mpvaVxKmc1,90.0
987,Here's a brigade of puppers. All look very prepared for whatever happens next. 80/80 https://t.co/0eb7R1Om12,80.0
1006,"From left to right:\nCletus, Jerome, Alejandro, Burp, &amp; Titson\nNone know where camera is. 45/50 would hug all at once https://t.co/sedre1ivTK",50.0
1069,Here is a whole flock of puppers. 60/50 I'll take the lot https://t.co/9dpcw6MdWa,50.0
1136,Happy Wednesday here's a bucket of pups. 44/40 would pet all at once https://t.co/HppvrYuamZ,40.0
1281,Someone help the girl is being mugged. Several are distracting her while two steal her shoes. Clever puppers 121/110 https://t.co/1zfnTJLt55,110.0
1300,This is Darrel. He just robbed a 7/11 and is in a high speed police chase. Was just spotted by the helicopter 10/10 https://t.co/7EsP8LmSp5,11.0


In [230]:
df_ta.loc[909,'rating_denominator'] = 10
df_ta.loc[837, 'rating_denominator'] = 10
df_ta.loc[939, 'rating_denominator'] = 10
df_ta.loc[617, 'rating_denominator'] = 10
df_ta.loc[1300, 'rating_denominator'] = 10

In [231]:
df_ta[['text', 'rating_numerator']].query('rating_numerator < 8');
#lower values look ok.

In [232]:
df_ta[['text', 'rating_denominator']].query('rating_denominator < 10') 

,text,rating_denominator


In [233]:
df_ta.loc[402,'rating_denominator'] = np.nan
df_ta.loc[1885, 'rating_denominator'] = 10

<a id='testq2'></a>
#### Test issues:

In [235]:
df_ta[['text', 'rating_numerator']].query('rating_numerator > 15') 

,text,rating_numerator
341,The floofs have been released I repeat the floofs have been released. 84/70 https://t.co/NIYC820tmd,84.0
716,Why does this never happen at my front door... 165/150 https://t.co/HmwrdfEfUE,165.0
772,This is Atticus. He's quite simply America af. 1776/10 https://t.co/GRXwMxLBkh,1776.0
875,Say hello to this unbelievably well behaved squad of doggos. 204/170 would try to pet all at once https://t.co/yGQI3He3xv,204.0
964,Happy Saturday here's 9 puppers on a bench. 99/90 good work everybody https://t.co/mpvaVxKmc1,99.0
987,Here's a brigade of puppers. All look very prepared for whatever happens next. 80/80 https://t.co/0eb7R1Om12,80.0
1006,"From left to right:\nCletus, Jerome, Alejandro, Burp, &amp; Titson\nNone know where camera is. 45/50 would hug all at once https://t.co/sedre1ivTK",45.0
1069,Here is a whole flock of puppers. 60/50 I'll take the lot https://t.co/9dpcw6MdWa,60.0
1136,Happy Wednesday here's a bucket of pups. 44/40 would pet all at once https://t.co/HppvrYuamZ,44.0
1281,Someone help the girl is being mugged. Several are distracting her while two steal her shoes. Clever puppers 121/110 https://t.co/1zfnTJLt55,121.0


In [234]:
df_ta[['text', 'rating_denominator']].query('rating_denominator < 10 or rating_denominator > 10') 

,text,rating_denominator
341,The floofs have been released I repeat the floofs have been released. 84/70 https://t.co/NIYC820tmd,70.0
716,Why does this never happen at my front door... 165/150 https://t.co/HmwrdfEfUE,150.0
875,Say hello to this unbelievably well behaved squad of doggos. 204/170 would try to pet all at once https://t.co/yGQI3He3xv,170.0
964,Happy Saturday here's 9 puppers on a bench. 99/90 good work everybody https://t.co/mpvaVxKmc1,90.0
987,Here's a brigade of puppers. All look very prepared for whatever happens next. 80/80 https://t.co/0eb7R1Om12,80.0
1006,"From left to right:\nCletus, Jerome, Alejandro, Burp, &amp; Titson\nNone know where camera is. 45/50 would hug all at once https://t.co/sedre1ivTK",50.0
1069,Here is a whole flock of puppers. 60/50 I'll take the lot https://t.co/9dpcw6MdWa,50.0
1136,Happy Wednesday here's a bucket of pups. 44/40 would pet all at once https://t.co/HppvrYuamZ,40.0
1281,Someone help the girl is being mugged. Several are distracting her while two steal her shoes. Clever puppers 121/110 https://t.co/1zfnTJLt55,110.0
1392,IT'S PUPPERGEDDON. Total of 144/120 ...I think https://t.co/ZanVtAtvIq,120.0


<a id='defq3'></a>
#### Defining issue: Compare hashtag data from json source and from text column, add missing values to df_ta

Hashtag data has been extracted from Json data, compare with hashtags in the text column and add missing values to hashtag column

In [295]:
hashtaglist = df_ta.hashtags.value_counts()
tweet_id_hashtags = df_ta[['hashtags', 'tweet_id']].query('hashtags in @hashtaglist.index').tweet_id


In [296]:
tweet_id_text = df_ta[df_ta.text.str.contains('#')].tweet_id
#8 hashtags more than I extracted from the json file
#rename jpg_url_json into jpg_url_short

def find_missing_hashtags (tweet_ids_text, tweet_ids_hashtags):
    
        dif1 = set(tweet_ids_text).difference(tweet_ids_hashtags)
        dif2 = set(tweet_ids_hashtags).difference(tweet_ids_text)
        
        return list(dif1)+ list(dif2)

update_hashtags = find_missing_hashtags(tweet_id_text, tweet_id_hashtags)    
    
    
    

In [297]:
df_ta[['tweet_id','text', 'hashtags']].query('tweet_id in @update_hashtags')

,tweet_id,text,hashtags


In [294]:
##manually change: 881268444196462592 #Canada, 874012996292530176 #PrideMonth, all others are BarkWeek Hashtags

df_ta['hashtags'] = df_ta[['tweet_id', 'hashtags']].mask(df_ta['tweet_id'] == 881268444196462592, "Canada").hashtags
df_ta['hashtags'] = df_ta[['tweet_id', 'hashtags']].mask(df_ta['tweet_id'] == 874012996292530176, "PrideMonth").hashtags

##run update_hashtags again to get only 'BarkWeek' hashtags and update
for i in update_hashtags:
    df_ta ['hashtags'] = df_ta[['tweet_id', 'hashtags']].mask(df_ta['tweet_id'] == i, "BarkWeek").hashtags

### Appendix:

Secure authorisation key outside of notebook:

http://veekaybee.github.io/2020/02/25/secrets/

https://pypi.org/project/python-dotenv/

http://docs.tweepy.org/en/latest/getting_started.html#api

https://www.w3resource.com/python-exercises/file/python-io-exercise-9.php

https://stackoverflow.com/questions/17322208/multiple-try-codes-in-one-block
